## Étude

In [8]:
import numpy as np
import random
import cv2

#### NPS (Non-parametric sampling)

Algorithme:

In [5]:
import cv2
import numpy as np

def growImage(sample_image, image, window_size):
    max_error = 0.3
    while not myCountNonZero(image) == image.shape[0] * image.shape[1]:
        pixel_list = getUnfilledNeighbours(image)
        if not pixel_list:
            break
        progress = 0
        for pixel in pixel_list:
            cv2.imshow("progress", image)
            cv2.waitKey(1)
            print(f"current pixel : {pixel[0]}x, {pixel[1]} y")
            window = np.array(getNeighbourWindow(image, pixel, window_size))
            matches = findMatches(sample_image, window, window_size)
            if matches:
                best_match = matches[0]
                for ch in range(3):
                    image[pixel[0], pixel[1], ch] = sample_image[best_match[0], best_match[1], ch]
            cv2.destroyAllWindows()
    cv2.imwrite("thisresult.png", np.ciel(image * 255))


def findMatches(sample_image, window, window_size):
    error_threshold = 0.3

    valid_mask = []
    columns = []
    for i in range(len(window)):
        columns = []
        for j in range(len(window)):
            if(window[i, j, :].any()):
                columns.append(1)
            else:
                columns.append(0)
        valid_mask.append(columns)

    sample_R, sample_G, sample_B = cv2.split(sample_image)
    template_R, template_G, template_B = cv2.split(window)
    sample_R = sample_R.astype(np.float32)
    sample_G = sample_G.astype(np.float32)
    sample_B = sample_B.astype(np.float32)
    
    template_R = template_R.astype(np.float32)
    template_G = template_G.astype(np.float32)
    template_B = template_B.astype(np.float32)

    dimensions = (sample_image.shape[0], sample_image.shape[1])
    result_R, result_G, result_B = np.zeros(dimensions), np.zeros(dimensions), np.zeros(dimensions)

    resR = cv2.matchTemplate(sample_R, template_R, cv2.TM_SQDIFF, result_R)
    resG = cv2.matchTemplate(sample_G, template_G, cv2.TM_SQDIFF, result_G)
    resB = cv2.matchTemplate(sample_B, template_B, cv2.TM_SQDIFF, result_B)

    result = (resR + resG + resB) / 3

    print(f"result :{np.min(result)}")

    resultList = []
    for i in range(sample_image.shape[0] - window_size):
        for j in range(sample_image.shape[1] - window_size):
            if(result[i, j] <= np.min(result)) * (1 + error_threshold):
                resultList.append((i + window_size // 2, j + window_size // 2))
    print(resultList)
    return(resultList)

def get_gaussian2D(window_size, sigma):
    x, y = np.meshgrid(np.arange(window_size), np.arange(window_size) + 1)
    x = x - window_size//2
    y = y - window_size//2
    return 1/(2*np.pi*sigma**2) * np.exp(-(x**2 + y**2)/(2*sigma**2))

def getNeighbourWindow(image, pixel, window_size):
    window = []
    for i in range(max(0, pixel[0] - window_size // 2), min(pixel[0] + window_size // 2, image.shape[0]) + 1):
        rows = []
        for j in range(max(0, pixel[1] - window_size // 2), min(pixel[1] + window_size // 2, image.shape[1]) + 1):
            colors = []
            for k in range(3):
                colors.append(image[i, j, k])
            rows.append(colors)
        window.append(rows)
    return window

# recherche des pixels qui sont voisins a l'extremité de l'image
def getUnfilledNeighbours(image):
    result = []
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if((image[i, j, 0] == image[i, j, 1] == image[i, j, 2] == 0)):
                try:
                    if(image[i-1 : i+2, j-1 : j+2, :].any()):
                        result.append((i, j))
                except IndexError:
                    print("pixel n'existe pas")
    return result

def myCountNonZero(image):
    result = 0
    for j in range(image.shape[0]):
        for k in range(image.shape[1]):
            if(not image[j, k, :].any()):
                continue
            else:
                result += 1
    return result // 3

window_size = 4
sample_image = cv2.imread(".\\128x128\\brick\\Brick_06-128x128.png") / 255
image = np.zeros((sample_image.shape[0], sample_image.shape[1], 3))

image[sample_image.shape[0] // 2 - 6 : sample_image.shape[0] // 2 + 7, sample_image.shape[1] // 2 - 6 : sample_image.shape[1] // 2 + 7, :] = sample_image[sample_image.shape[0] // 2 - 6 : sample_image.shape[0] // 2 + 7, sample_image.shape[1] // 2 - 6 : sample_image.shape[1] // 2 + 7, :]

image = growImage(sample_image, image, window_size)

cv2.imwrite("sample.png", np.ciel(image * 255))
cv2.waitKey(0)
cv2.destroyAllWindows()

current pixel : 57x, 57 y
result :2.1599998474121094
[(61, 15)]
current pixel : 57x, 58 y
result :2.0393083095550537
[(118, 124)]
current pixel : 57x, 59 y
result :1.8028253316879272
[(118, 125)]
current pixel : 57x, 60 y
result :1.285941481590271
[(63, 13)]
current pixel : 57x, 61 y
result :1.2587412595748901
[(63, 14)]
current pixel : 57x, 62 y
result :1.2778984308242798


### Patch-based TS

In [ ]:
! ..\patch-based\texture_synthesis.py --image_path ../patch-based/data/input9.png --block_size 30 --num_block 14 --mode Best